# Wide and Deep Model for Movie Recommendation
```
TODO details about wide-deep
```

[Wide-deep model](https://arxiv.org/abs/1606.07792) for a recommender system, and

### Prerequisite
* tensorflow (version 1.10 or higher)

In [1]:
import sys
sys.path.append("../../")

import itertools
import os
import shutil

import tensorflow as tf
import pandas as pd

from reco_utils.common import tf_utils
from reco_utils.dataset import movielens
from reco_utils.dataset.python_splitters import python_random_split
from reco_utils.evaluation.python_evaluation import (
    rmse, mae, rsquared, exp_var,
    map_at_k, ndcg_at_k, precision_at_k, recall_at_k
)

In [2]:
from tensorflow.python.client import device_lib

print("Tensorflow Version:", tf.__version__)

devices = device_lib.list_local_devices()
[x.name for x in devices]

Tensorflow Version: 1.12.0


['/device:CPU:0', '/device:GPU:0']

In [9]:
# top k items to recommend
TOP_K = 10

# Select Movielens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '100k'

### Data loading

Download [MovieLens](https://grouplens.org/datasets/movielens/) data and split train / test set

In [10]:
data = movielens.load_pandas_df(
    size=MOVIELENS_DATA_SIZE,
    header=['UserId','MovieId','Rating','Timestamp'],
    # TODO For now, not using genres YET
    load_genres=False
)
print(data.head())

train, test = python_random_split(data, ratio=0.75, seed=123)

X_train = train.copy()
y_train = X_train.pop('Rating')
X_test = test.copy()
y_test = X_test.pop('Rating')

# Distinct users and items (use for feature embedding)
user_list = data['UserId'].unique()
item_list = data['MovieId'].unique()
print("#Users =", len(user_list))
print("#Items =", len(item_list))

   UserId  MovieId  Rating  Timestamp
0     196      242     3.0  881250949
1     186      302     3.0  891717742
2      22      377     1.0  878887116
3     244       51     2.0  880606923
4     166      346     1.0  886397596
#Users = 943
#Items = 1682


### Modeling

* `wide` - Linear model
* `deep` - DNN model
* `wide_deep` - Linear combination of the linear and DNN models

In [12]:
""" Hyper parameters
"""
# Model checkpoints folder
MODEL_DIR = './models'

MODEL_TYPE = 'wide'

BATCH_SIZE = 32
NUM_EPOCHS = 10

LINEAR_OPTIMIZER = 'SGD'
LINEAR_OPTIMIZER_LR = 0.001
DNN_OPTIMIZER = 'Adagrad'
DNN_OPTIMIZER_LR = 0.001
DNN_HIDDEN_UNITS = [256, 256, 128]
DNN_DROPOUT = None
DNN_BATCH_NORM = False

# Rule of thumb for embedding_dimensions =  number_of_categories ** 0.25
USER_EMBEDDING_DIM = int(len(user_list) ** 0.25)
ITEM_EMBEDDING_DIM = int(len(item_list) ** 0.25)

### Feature embedding

Wide and deep model utilizes two different types of feature set: 1) a wide set of cross-producted features to capture how the co-occurrence of a query-item feature pair correlates with the target label or rating, and 2) a deep, lower-dimensional embedding vectors for every query and item.

In [13]:
wide_columns = []
deep_columns = []

if MODEL_TYPE == 'wide' or MODEL_TYPE == 'wide_deep':
    wide_columns = tf_utils.build_feature_columns(
        'wide', user_list, item_list, 'UserId', 'MovieId'
    )
if MODEL_TYPE == 'deep' or MODEL_TYPE == 'wide_deep':
    deep_columns = tf_utils.build_feature_columns(
        'deep', user_list, item_list, 'UserId', 'MovieId', None, 'Timestamp',
        USER_EMBEDDING_DIM, ITEM_EMBEDDING_DIM, 0
    )
    print("Embedding {} users to {}-dim vector".format(len(user_list), USER_EMBEDDING_DIM))
    print("Embedding {} items to {}-dim vector".format(len(item_list), ITEM_EMBEDDING_DIM))

# To check features
# TODO
# features = tf.parse_example(..., features=make_parse_example_spec(columns))
# dense_tensor = input_layer(features, columns)

In [14]:
# TODO set run tf config if needed

if MODEL_TYPE == 'wide':
    model = tf.estimator.LinearRegressor(  # LinearClassifier(
        model_dir=MODEL_DIR,
        feature_columns=wide_columns,
        optimizer=tf_utils.build_optimizer(LINEAR_OPTIMIZER, LINEAR_OPTIMIZER_LR)
    )
elif MODEL_TYPE == 'deep':
    model = tf.estimator.DNNRegressor(  # DNNClassifier(
        model_dir=MODEL_DIR,
        feature_columns=deep_columns,
        hidden_units=DNN_HIDDEN_UNITS,
        optimizer=tf_utils.build_optimizer(DNN_OPTIMIZER, DNN_OPTIMIZER_LR),
        dropout=DNN_DROPOUT,
        batch_norm=DNN_BATCH_NORM
    )
elif MODEL_TYPE == 'wide_deep':
    model = tf.estimator.DNNLinearCombinedRegressor(  # DNNLinearCombinedClassifier(
        model_dir=MODEL_DIR,
        # wide settings
        linear_feature_columns=wide_columns,
        linear_optimizer=tf_utils.build_optimizer(LINEAR_OPTIMIZER, LINEAR_OPTIMIZER_LR),
        # deep settings
        dnn_feature_columns=deep_columns,
        dnn_hidden_units=DNN_HIDDEN_UNITS,
        dnn_optimizer=tf_utils.build_optimizer(DNN_OPTIMIZER, DNN_OPTIMIZER_LR),
        dnn_dropout=DNN_DROPOUT,
        batch_norm=DNN_BATCH_NORM
    )
else:
    raise ValueError("Model type should be either 'wide', 'deep', or 'wide_deep'")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './models', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002516D7C6F98>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


### Training

In [25]:
class _TrainLogHook(tf.train.SessionRunHook):
    """Training hook to log the given metrics for every n iteration"""

# TODO Add estimation like...
# def __init__(self, model_fn, params, input_fn, checkpoint_dir,
#                  every_n_secs=None, every_n_steps=None):
#         self._iter_count = 0
#         self._estimator = tf.estimator.Estimator(
#             model_fn=model_fn,
#             params=params,
#             model_dir=checkpoint_dir
#         )
#         self._input_fn = input_fn
#         self._timer = tf.train.SecondOrStepTimer(every_n_secs, every_n_steps)
#         self._should_trigger = False

#     def begin(self):
#         self._timer.reset()
#         self._iter_count = 0

#     def before_run(self, run_context):
#         self._should_trigger = self._timer.should_trigger_for_step(self._iter_count)

#     def after_run(self, run_context, run_values):
#         if self._should_trigger:
#             self._estimator.evaluate(
#                 self._input_fn
#             )
#             self._timer.update_last_triggered_step(self._iter_count)
#         self._iter_count += 1


#     def __init__(self, metrics, every_n_iter=100):
#         """
#             every_n_iter (int): Logging frequency (iterations)
#         """
#         if every_n_iter <= 0:
#             raise ValueError("Log frequency should be greater than 0 iteration")
            
#         self.metrics = metrics
#         self.every_n_iter = every_n_iter
#         self.iter = 0

#     def before_run(self, run_context):
#         self.iter += 1
#         if (self.iter % self.every_n_iter) == 0:
#             return tf.train.SessionRunArgs(self.metrics)
#         else:
#             return None

#     def after_run(self, run_context, run_values):
#         metrics_value = run_values.results
#         if metrics_value is not None:
#             print("{}: {}".format(self.metrics, metrics_value))

In [33]:
train_input_fn = tf.estimator.inputs.pandas_input_fn(
    x=X_train,
    y=y_train,
    batch_size=BATCH_SIZE,
    num_epochs=NUM_EPOCHS,
    shuffle=True,
    num_threads=1
)

# Add additional evaluation metrics
metrics_fn = tf_utils.eval_metrics('rmse')
model = tf.contrib.estimator.add_metrics(model, metrics_fn)

monitors = [tf.contrib.learn.monitors.ValidationMonitor(
    input_fn=train_input_fn,
    every_n_steps=100
)]
hooks = tf.contrib.learn.monitors.replace_monitors_with_hooks(monitors, model)

# validation_metrics = {
#     'rmse': tf.contrib.learn.MetricSpec(
#         metric_fn=metrics_fn,
#         prediction_key='predictions'
#     )
# }

tf.logging.set_verbosity(tf.logging.INFO)
# logging_hook = tf.train.LoggingTensorHook(validation_metrics, every_n_iter=10, at_end=True)


#   tf.add_to_collection('losses', cross_entropy_mean)

#   # The total loss is defined as the cross entropy loss plus all of the weight
#   # decay terms (L2 loss).
#   return tf.add_n(tf.get_collection('losses'), name='total_loss')

#     tf.train.LoggingTensorHook(
#         tensors=tf.get_default_graph().get_tensor_by_name('rmse'),
#         every_n_iter=100,
#         at_end=True
#     )

model.train(input_fn=train_input_fn, hooks=hooks)

INFO:tensorflow:Using config: {'_model_dir': './models', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002516C2F42B0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorfl

KeyboardInterrupt: 

### Testing

We predict the ratings by using the wide-deep model we trained. Finally, we also generate top-k movie recommentation for each user and test the performance.

In [ ]:
test_input_fn = tf.estimator.inputs.pandas_input_fn(
    x=test_x,
    y=test_y,
    batch_size=BATCH_SIZE,
    num_epochs=1,
    shuffle=False
)

pred_list = [p['predictions'][0] for p in list(model.predict(input_fn=test_input_fn))]
predictions = test.copy()
predictions['prediction']  = pd.Series(pred_list).values
print(predictions.head())

1. Item rating prediction

In [ ]:
cols = {
    'col_user': 'UserId',
    'col_item': 'MovieId',
    'col_rating': 'Rating',
    'col_prediction': 'prediction'
}

predictions.drop('Rating', axis=1, inplace=True)

eval_rmse = rmse(test, predictions, **cols)
eval_mae = mae(test, predictions, **cols)
eval_rsquared = rsquared(test, predictions, **cols)
eval_exp_var = exp_var(test, predictions, **cols)

print("RMSE:\t\t%f" % eval_rmse,
      "MAE:\t\t%f" % eval_mae,
      "rsquared:\t%f" % eval_rsquared,
      "exp var:\t%f" % eval_exp_var, sep='\n')

In [ ]:
# Compare w/ TF Evaluation
result = model.evaluate(input_fn=test_input_fn, steps=None)
print("\nEvaluation:", result)

2. Recommend k items

1) Remove seen items and 2) add timestamp info

In [ ]:
# Get the cross join of all user-item pairs and score them.
user_item_col = ['UserId', 'MovieId']
user_item_list = list(itertools.product(user_list, item_list))
users_items = pd.DataFrame(user_item_list, columns=user_item_col)
print("Before excude seen items:", len(users_items))

# Remove seen items (items in the train set)
users_items_exclude_train = users_items.loc[
    ~users_items.set_index(user_item_col).index.isin(train.set_index(user_item_col).index)
]
print("After excude seen items:", len(users_items_exclude_train))

# Add timestamp info
users_items_exclude_train = pd.merge(test, users_items_exclude_train,
                                     on=user_item_col, how='outer')
users_items_exclude_train.drop('Rating', axis=1, inplace=True)
users_items_exclude_train.fillna(test['Timestamp'].max(), inplace=True) 
print(users_items_exclude_train.head())

In [ ]:
reco_input_fn = tf.estimator.inputs.pandas_input_fn(
    x=users_items_exclude_train,
    batch_size=BATCH_SIZE,
    num_epochs=1,
    shuffle=False
)

reco = list(model.predict(input_fn=reco_input_fn))
reco_list = [p['predictions'][0] for p in reco]
users_items_exclude_train['prediction']  = pd.Series(reco_list).values
users_items_exclude_train.head()


In [ ]:
k = 10
eval_map = map_at_k(test, users_items_exclude_train, k=k, **cols)
eval_ndcg = ndcg_at_k(test, users_items_exclude_train, k=k, **cols)
eval_precision = precision_at_k(test, users_items_exclude_train, k=k, **cols)
eval_recall = recall_at_k(test, users_items_exclude_train, k=k, **cols)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

Export and reload model

In [ ]:
EXPORT_DIR = './saved_model'
os.makedirs(EXPORT_DIR, exist_ok=True)

# maybe use build_supervised_input_receiver_fn_from_input_fn
train_rcvr_fn = tf.contrib.estimator.build_supervised_input_receiver_fn_from_input_fn(
    train_input_fn
)

serve_rcvr_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(
    tf.feature_column.make_parse_example_spec(feat_columns)
)

rcvr_fn_map = {
    tf.estimator.ModeKeys.TRAIN: train_rcvr_fn,
    tf.estimator.ModeKeys.EVAL: train_rcvr_fn,
    tf.estimator.ModeKeys.PREDICT: serve_rcvr_fn
}

export_dir = tf.contrib.estimator.export_all_saved_models(
    model,
    export_dir_base=EXPORT_DIR,
    input_receiver_fn_map=rcvr_fn_map
)


In [ ]:
saved_model = tf.contrib.estimator.SavedModelEstimator(export_dir)

result = saved_model.evaluate(input_fn=test_input_fn, steps=None)
print(result)

In [ ]:
def predict_input_fn():
    examples = []
    for index, row in test_x.iterrows():
        feature = {}
        for col, value in row.iteritems():
            feature[col] =  tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(str(value), encoding='ascii')]))
        example = tf.train.Example(
            features=tf.train.Features(
                feature=feature
            )
        )
        examples.append(example.SerializeToString())
    return {'inputs': tf.constant(examples)}
       
# def predict_input_fn():
#     example = tf.train.Example()
#     example.features.feature['UserId'].bytes_list.value.extend(['496'])
#     example.features.feature['MovieId'].bytes_list.value.extend(['136'])
#     return {'inputs': tf.constant([example.SerializeToString()])}


In [ ]:



# Convert input data into serialized Example strings.


# features = tf.parse_example(
#     serialized=serialized_examples,
#     features=make_parse_example_spec(feature_columns))
# predictions_dict = next(prediction)
# predictions_dict
# pred_input_fn = tf.estimator.inputs.pandas_input_fn(
#     x=test_x,
#     num_epochs=1,
#     shuffle=False
# )

predictions = saved_model.predict(predict_input_fn)


In [ ]:
# Cleanup
shutil.rmtree(EXPORT_DIR)
shutil.rmtree(MODEL_DIR)

In [ ]:
# https://github.com/MtDersvan/tf_playground/blob/master/wide_and_deep_tutorial/wide_and_deep_export_r1.3.ipynb
https://github.com/Azure/MachineLearningNotebooks/tree/master/how-to-use-azureml/training-with-deep-learning